In [1]:
import os
import sqlite3
import json
import random
import seaborn as sns
from tqdm.notebook import tqdm

import torch
import numpy as np
import datasets
import transformers
import wandb
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, DataCollator

2022-09-30 06:25:05.338948: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64
2022-09-30 06:25:05.338989: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
model_card = "google/electra-small-discriminator"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_card)

In [4]:
sst2 = datasets.load_dataset('glue', 'qqp')
sst2

Reusing dataset glue (/home/johnny/.cache/huggingface/datasets/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question1', 'question2', 'label', 'idx'],
        num_rows: 363846
    })
    validation: Dataset({
        features: ['question1', 'question2', 'label', 'idx'],
        num_rows: 40430
    })
    test: Dataset({
        features: ['question1', 'question2', 'label', 'idx'],
        num_rows: 390965
    })
})

In [5]:
def tokenize_function(examples, field='question1'):
    return tokenizer(examples[field], padding=False, truncation=True)
tokenized_sst2 = sst2.map(tokenize_function, batched=True)
lengths = [len(i) for i in tokenized_sst2['train']['input_ids']]
max_length = np.quantile(lengths, 0.9)
# max_length = np.max(lengths)
# max_length = 124
max_length

  0%|          | 0/364 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-c119dbf8de4dd4a1.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-30dbb76d4ef918aa.arrow


24.0